In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
filepath = "movies_metadata.csv"


In [3]:
# Create a dataframe from csv; drop all rows with no movie title.
df_raw = pd.read_csv(filepath, delimiter=',')
df = df_raw.dropna(subset=['title'])

# User list comprehension to create a list of lists from Dataframe rows
list_of_rows = [list(row) for row in df.values]
type(list_of_rows)

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


list

In [4]:
print(len(list_of_rows))

45460


In [12]:
list_of_langs_dataframes = []
for row in list_of_rows:
             
        lang_list = eval(row[17])
        record_df = pd.DataFrame(lang_list)
        record_df['movie_id_number'] = row[5]
        list_of_langs_dataframes.append(record_df)

langs_table = pd.concat(list_of_langs_dataframes, sort=False)
langs_table.reset_index(inplace=True, drop=True)
langs_table

,iso_639_1,name,movie_id_number
0,en,English,862
1,en,English,8844
2,fr,Français,8844
3,en,English,15602
4,en,English,31357
...,...,...,...
53295,en,English,30840
53296,fa,فارسی,439050
53297,tl,,111109
53298,en,English,67758


In [33]:
languages_df = langs_table[['iso_639_1','name']]
language_iso_tbl = languages_df.drop_duplicates()
language_iso_tbl.rename(columns={'name':'language'}, inplace=True)
language_iso_tbl.reset_index()
language_iso_tbl.head()

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,iso_639_1,language
0,en,English
2,fr,Français
7,es,Español
11,de,Deutsch
14,ru,Pусский


In [34]:
language_junction_df = langs_table[['iso_639_1','movie_id_number']]
language_junction_df.rename(columns={'movie_id_number':'movie_id'}, inplace=True)
language_junction_df.reset_index()
language_junction_df.head()

C:\Users\Walte\.conda\envs\PythonData\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,iso_639_1,movie_id
0,en,862
1,en,8844
2,fr,8844
3,en,15602
4,en,31357


In [35]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['keyword_movie_junction',
 'keywords',
 'ratings',
 'movies_data',
 'lang_movie_junction',
 'spoken_languages']

In [37]:
language_iso_tbl.to_sql(name='spoken_languages', con=engine, if_exists='append', index=False)

In [38]:
language_junction_df.to_sql(name='lang_movie_junction', con=engine, if_exists='append', index=False)